In [1]:
from bs4 import BeautifulSoup
import requests
import os
from tqdm.notebook import tqdm

In [2]:
BEGIN_DATE = '2023-12-10'
END_DATE = '2023-12-14'
PAGE_SIZE = '100'
START = '0'
PAGE = 11
TYPE = '&classification-computer_science=y'
# TYPE = '&classification-mathematics=y'
TYPE = '&classification-physics=y'

PATH = 'arxiv_pdfs_phi'

In [3]:
path_list = []

for START in [str(i * int(PAGE_SIZE)) for i in range(PAGE)]:
    
    url = f'https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=&terms-0-field=title{TYPE}&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date={BEGIN_DATE}&date-to_date={END_DATE}&date-date_type=submitted_date_first&abstracts=hide&size={PAGE_SIZE}&order=-announced_date_first&start={START}'
    print(f'url:{url}')

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    papers = soup.find_all('li', class_='arxiv-result')

    pdf_links = [paper.find('a', string='pdf')['href'] for paper in papers if paper.find('a', string='pdf')]

    print(f"start:{START} success:{len(pdf_links)}")
    
    path_list += pdf_links
    
    if len(pdf_links) == 0:
        break

print(f'total:{len(path_list)}')

url:https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date=2023-12-10&date-to_date=2023-12-14&date-date_type=submitted_date_first&abstracts=hide&size=100&order=-announced_date_first&start=0
start:0 success:100
url:https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date=2023-12-10&date-to_date=2023-12-14&date-date_type=submitted_date_first&abstracts=hide&size=100&order=-announced_date_first&start=100
start:100 success:99
url:https://arxiv.org/search/advanced?advanced=1&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-include_cross_list=exclude&date-year=&date-filter_by=date_range&date-from_date=2023-12-10&date-to_date=20

In [ ]:
import requests
from requests.exceptions import RequestException
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import time
import threading

def download_pdf(pdf_url, path, total_size, max_retries=3, timeout=10, update_progress=None):
    pdf_name = pdf_url.split('/')[-1]
    file_path = os.path.join(path, f'{pdf_name}.pdf')

    attempts = 0
    while attempts < max_retries:
        try:
            with requests.get(pdf_url, stream=True, timeout=timeout) as r:
                r.raise_for_status()
                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            if update_progress:
                update_progress()
            return True
        except RequestException as e:
            attempts += 1
            time.sleep(1)
            if attempts == max_retries:
                if update_progress:
                    update_progress()
                return False

def progress_monitor(total_tasks):
    progress = tqdm(total=total_tasks, desc="downloading", leave=True)
    while not progress_monitor.finished:
        progress.n = progress_monitor.completed_tasks
        progress.refresh()
        time.sleep(0.5)
    progress.n = progress_monitor.completed_tasks
    progress.refresh()
    progress.close()

def download_pdfs_concurrently(pdf_links, path, num_threads=10):
    os.makedirs(path, exist_ok=True)
    progress_monitor.completed_tasks = 0
    progress_monitor.finished = False

    def update_progress():
        progress_monitor.completed_tasks += 1

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(download_pdf, url, path, 0, update_progress=update_progress) for url in pdf_links]
        
        monitor_thread = threading.Thread(target=progress_monitor, args=(len(pdf_links),))
        monitor_thread.start()

        for future in as_completed(futures):
            pass

    progress_monitor.finished = True
    monitor_thread.join()
    

download_pdfs_concurrently(path_list, PATH)


downloading:   0%|          | 0/498 [00:00<?, ?it/s]